In [19]:
import re
import requests

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

In [2]:
tf.__version__

'2.3.0'

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  print("Invalid device or cannot modify virtual devices once initialized.")
  pass

In [4]:
def preprocess(string):
    string = string.lower()
    return re.sub(r"[\W_]+", " ", string)

,Unnamed: 0,id,img,label,text,embeds
0,0,42953,data/img/42953.png,0,its their character not their color that matters,[-3.78659666e-01 7.68264011e-02 1.49976397e+...
1,1,23058,data/img/23058.png,0,don t be afraid to love again everyone is not ...,[-9.69222248e-01 1.55724302e-01 1.42747843e+...
2,2,13894,data/img/13894.png,0,putting bows on your pet,[-1.04558063e+00 3.86801511e-01 5.03332555e-...
3,3,37408,data/img/37408.png,0,i love everything and everybody except for squ...,[-9.41388369e-01 8.97736013e-01 -5.95757961e-...
4,4,82403,data/img/82403.png,0,everybody loves chocolate chip cookies even hi...,[-7.81644404e-01 1.24291909e+00 -6.73934758e-...
...,...,...,...,...,...,...
8495,8495,10423,data/img/10423.png,1,nobody wants to hang auschwitz me,[-0.4600818 0.33925965 1.3114299 -0.363946...
8496,8496,98203,data/img/98203.png,1,when god grants you a child after 20 years of ...,[-9.21496630e-01 -1.07220873e-01 2.52688944e-...
8497,8497,36947,data/img/36947.png,1,gays on social media equality body positivity ...,[-8.43962431e-01 2.14604035e-01 1.47008568e-...
8498,8498,16492,data/img/16492.png,1,having a bad day you could be a siamese twin a...,[-1.17862451e+00 5.30087173e-01 5.42090654e-...


In [ ]:
train_df = pd.read_json('data/train.jsonl', lines = True)
train_df["img"] = "data/" + train_df["img"]
train_df["text"] = train_df["text"].apply(preprocess)

In [ ]:
val_df = pd.read_json('data/dev.jsonl', lines = True)
val_df["img"] = "data/" + val_df["img"]
val_df["text"] = val_df["text"].apply(preprocess)

In [5]:
val_df = pd.read_csv('data/val_df.csv')
val_df

,Unnamed: 0,id,img,label,text,embeds
0,0,8291,data/img/08291.png,1,white people is this a shooting range,[ 2.21438572e-01 1.22081435e+00 -5.13000607e-...
1,1,46971,data/img/46971.png,1,bravery at its finest,[-6.67330563e-01 -1.46951422e-01 9.92148399e-...
2,2,3745,data/img/03745.png,1,your order comes to 37 50 and your white privi...,[-1.04087675e+00 -2.56364256e-01 -5.20168185e-...
3,3,83745,data/img/83745.png,1,it is time to send these parasites back to the...,[ 9.67570692e-02 3.39410365e-01 3.22937548e-...
4,4,80243,data/img/80243.png,1,mississippi wind chime,[-5.05479038e-01 2.97429204e-01 -2.10287809e-...
...,...,...,...,...,...,...
495,495,83675,data/img/83675.png,0,i m gonna be like phelps one day,[-0.5706553 0.5400016 0.8655728 -0.581885...
496,496,37198,data/img/37198.png,0,when you re so relaxed you can feel yourself g...,[-8.09241056e-01 -4.33698654e-01 1.75170040e+...
497,497,48670,data/img/48670.png,0,look at this sandwich maker club i found on wi...,[-2.62110740e-01 5.62873244e-01 -6.70083880e-...
498,498,9863,data/img/09863.png,0,diverse group of women,[ 4.50953729e-02 2.20607236e-01 7.69905329e-...


In [6]:
img_width = 64
img_height = 64

In [11]:
def build_model():
    input_img = tf.keras.layers.Input(
        shape = (img_width, img_height, 3), name = "image"
    )
    
    embeds = tf.keras.layers.Input(name = "embeds", shape = (768,))

    extractor = tf.keras.applications.EfficientNetB0(include_top = False, \
                                                 input_tensor = input_img, weights = "imagenet")
    
    extractor.trainable = False

    x = tf.keras.layers.GlobalAveragePooling2D()(extractor.output)
    
    x = tf.keras.layers.concatenate([x, embeds])
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(128)(x)
    
    x = tf.keras.layers.Dropout(0.2)(x)
    
    out = tf.keras.layers.Dense(1, activation = 'sigmoid', name = "label")(x)

    model = tf.keras.models.Model([input_img, embeds], out)

    model.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ["accuracy"])
    return model

In [12]:
model = build_model()
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 64, 64, 3)    0           image[0][0]                      
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 64, 64, 3)    7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 65, 65, 3)    0           normalization_2[0][0]            
_______________________________________________________________________________________

In [13]:
def encode_single_sample(img_path, label, embeds):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    return {"image": img, "label": label, "embeds": embeds}

In [14]:
class HatefulMemes(tf.keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, input_img_paths, label, text):
        self.batch_size = batch_size
        self.input_img_paths = input_img_paths
        self.text = text
        self.label = label

    def __len__(self):
        return len(self.input_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_label = self.label[i : i + self.batch_size]
        batch_text = self.text[i : i + self.batch_size]
        x = np.zeros((self.batch_size,) + (img_height, img_width) + (3,), dtype = "float32")
        y = np.zeros((self.batch_size,) + (768,), dtype = "float32")
        z = np.zeros((self.batch_size,), dtype = "uint8")
        
        for i,j,k,l in zip(range(self.batch_size), batch_input_img_paths, batch_label, batch_text):
            sample = encode_single_sample(j,k,l)
            x[i] = sample["image"].numpy().tolist()
            y[i] = sample["embeds"]
            z[i] = sample["label"]
        
        return [x,y], z

In [15]:
train_gen = HatefulMemes(16, train_df["img"].values.tolist(), train_df["label"].values.tolist(), train_df["embeds"].values.tolist())
val_gen = HatefulMemes(16, val_df["img"].values.tolist(), val_df["label"].values.tolist(), val_df["embeds"].values.tolist())

In [16]:
epochs = 100
early_stopping_patience = 3

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = "val_accuracy", patience = early_stopping_patience, restore_best_weights = True
)

# Train the model
history = model.fit(
    train_gen,
    validation_data = val_gen, 
    epochs = epochs,
    callbacks = [early_stopping],
)

ValueError: could not convert string to float: '[-3.78659666e-01  7.68264011e-02  1.49976397e+00 -5.24096668e-01\n -9.00999546e-01  1.11138475e+00  1.38201758e-01  1.12481856e+00\n -2.35449106e-01 -1.00472248e+00  2.89959796e-02 -1.52343810e-01\n  6.04895532e-01  2.65746713e-01 -9.11300063e-01  5.33768773e-01\n  6.93978190e-01 -3.74570131e-01  3.19228232e-01  6.43576026e-01\n -5.04141450e-02  4.90637213e-01 -6.97442532e-01  4.78632390e-01\n -1.01988256e-01 -6.41101673e-02  7.08310127e-01 -4.07322735e-01\n  4.31538582e-01  1.37420928e+00 -3.47858727e-01  4.48168457e-01\n -1.47088182e+00  4.14856344e-01 -4.63820606e-01 -1.05911219e+00\n  2.69176304e-01 -2.91562855e-01 -7.38245010e-01 -2.16756105e-01\n  1.85100678e-02 -3.22210491e-01  6.07826531e-01 -2.15646654e-01\n  7.07927942e-01 -4.98130798e-01  4.62596491e-02 -2.16891840e-01\n  2.45963246e-01 -2.52486080e-01 -4.09964025e-01 -5.77342790e-03\n -3.27808201e-01  1.24043727e+00  3.42830479e-01 -9.27890778e-01\n -6.76166177e-01  4.79108810